In [32]:
from nltk.stem.porter import PorterStemmer
from ast import literal_eval
    

def tokenize(text):
    stemmeren = PorterStemmer()
    lowercase = [tkn.lower() for tkn in text]
    
    stop_rm = [wd for wd in lowercase if not wd in STOPWORDSEN]
    stem = [stemmeren.stem(wd) for wd in stop_rm]
    return [j for j in stop_rm if j]

# load stop words list
stopWordsSet = sc.textFile("stop-word-list.txt").collect()
STOPWORDSEN = set(stopWordsSet)

with open('data_country.txt','r') as f:
    fd = f.read()
res = literal_eval(fd)
res = tokenize(res)
length = len(res)
# print(length)
# print(res)
    
def cal_lasting(ddl,launch):
    res = int(ddl) - int(launch)
    return list([res])
    
def dcTransformMapper(elem):
    if elem == 'False':
        return [0]
    else:
        return [1]
    
def splitMapper(elem):
    return htf.transform(elem)
    
from pyspark.sql import Row
from pyspark.ml.linalg import Vectors,DenseVector
import numpy as np
# from pyspark.mllib.feature import HashingTF
# from pyspark.mllib.feature import IDF
# from pyspark.mllib.regression import LabeledPoint
from pyspark.ml.feature import HashingTF,IDF,Tokenizer,RegexTokenizer


def f(x):
    rel = {}
    rel['final_status'] = str(x[4])
    rel['features'] = Vectors.dense(float(x[0]),float(x[1]),float(x[2]),float(x[3]))
    return rel

import csv
# remove first row(header)
trainKickstarter = sc.textFile("funding-successful-projects-on-kickstarter/train.csv")
trainHeader = trainKickstarter.first()
trainKickstarter = trainKickstarter.filter(lambda line: line != trainHeader).mapPartitions(lambda x: csv.reader(x))
# print(trainKickstarter.take(10))

# Data Transformations and filter lines with empty strings
trainKickstarter=trainKickstarter.map(lambda line: line[3:4]+dcTransformMapper(line[5])+cal_lasting(line[8],line[11])+line[12:14])
print(trainKickstarter.take(10))

# trainKickstarter = spark.createDataFrame(trainKickstarter).toDF('goal','country','d_c','lasting_time','backers','final_status')
# trainKickstarter.show(10)

# reTokenizer = RegexTokenizer(inputCol="country", outputCol="words", pattern="\\W")
# wordsData = reTokenizer.transform(trainKickstarter)
# wordsData.show(10)

# hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=11)
# featurizedData = hashingTF.transform(wordsData)
# featurizedData.show(10)

# idf = IDF(inputCol="rawFeatures", outputCol="features")
# idfModel = idf.fit(featurizedData)

# trainKickstarter = idfModel.transform(featurizedData)
# trainKickstarter.select("final_status", "features").show(10)

#---------------
# htf = HashingTF(length+200)

# hashedData = trainKickstarter.map(lambda line: line[0:1]+htf.transform(line[1]))
# print(hashedData.take(10))
# DenseVectors = hashedData.map(lambda vector: DenseVector(vector.toArray()))
# print(DenseVectors.take(10))
# print(trainKickstarter.take(10))

trainKickstarter=trainKickstarter.map(lambda p: Row(**f(p))).toDF()
# print(trainKickstarter)

print(trainKickstarter.show(10))
trainKickstarter.printSchema()




[['20', 1, 731276, '3', '1'], ['300', 1, 1453408, '2', '0'], ['30', 1, 863162, '0', '0'], ['500', 1, 2589010, '18', '1'], ['2000', 1, 2589339, '1', '0'], ['700', 1, 2351132, '14', '0'], ['250', 1, 2135692, '2', '0'], ['1000', 1, 2617709, '32', '1'], ['5000', 1, 2783499, '44', '0'], ['3500', 1, 2673080, '18', '0']]
+--------------------+------------+
|            features|final_status|
+--------------------+------------+
|[20.0,1.0,731276....|           1|
|[300.0,1.0,145340...|           0|
|[30.0,1.0,863162....|           0|
|[500.0,1.0,258901...|           1|
|[2000.0,1.0,25893...|           0|
|[700.0,1.0,235113...|           0|
|[250.0,1.0,213569...|           0|
|[1000.0,1.0,26177...|           1|
|[5000.0,1.0,27834...|           0|
|[3500.0,1.0,26730...|           0|
+--------------------+------------+
only showing top 10 rows

None
root
 |-- features: vector (nullable = true)
 |-- final_status: string (nullable = true)



In [26]:
# RandomForest
from pyspark.ml import Pipeline
from pyspark.ml.feature import IndexToString,StringIndexer,VectorIndexer
from pyspark.ml.classification import RandomForestClassifier
import time

startTime = time.time()

# Index labels, adding metadata to the label column.
# Fit on whole dataset to include all labels in index.
labelIndexer = StringIndexer(inputCol="final_status", outputCol="indexedLabel").fit(trainKickstarter)
# labelIndexer = StringIndexer().setInputCol("label").setOutputCol("indexedLabel").fit(trainTitanic)

# Automatically identify categorical features, and index them.
featureIndexer = VectorIndexer(inputCol="features", outputCol="indexedFeatures").fit(trainKickstarter)
# featureIndexer = VectorIndexer().setInputCol("features").setOutputCol("indexedFeatures").fit(trainTitanic)


# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = trainKickstarter.randomSplit([0.7, 0.3])

# Train a RandomForest model.
rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", numTrees=10)

# Convert indexed labels back to original labels.
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel",labels=labelIndexer.labels)


# Chain indexers and forest in a Pipeline
Pipeline =  Pipeline().setStages([labelIndexer, featureIndexer, rf, labelConverter])
# Train model.  This also runs the indexers.
PipelineModel = Pipeline.fit(trainingData)

endTime = time.time()
executeTime = endTime - startTime
print ('Execution Time For Random Forest = %s' % str(executeTime) + ' Seconds\n')

# Make predictions.
Predictions = PipelineModel.transform(testData)
Predictions.select("predictedLabel", "final_status").show(10)


preRel = Predictions.select("predictedLabel", "final_status", "features", "probability").collect()

print('''final_status --> prob = [prob1,prob2], predicted_final_status''')
for item in preRel:
    print(str(item['final_status'])+' -->prob= '+str(item['probability'])+',predicted_final_status: '+str(item['predictedLabel']))
 


Execution Time For Random Forest = 13.116832733154297 Seconds

+--------------+------------+
|predictedLabel|final_status|
+--------------+------------+
|             0|           1|
|             1|           1|
|             1|           1|
|             1|           1|
|             0|           0|
|             1|           1|
|             1|           1|
|             1|           1|
|             1|           1|
|             1|           1|
+--------------+------------+
only showing top 10 rows

final_status --> prob = [prob1,prob2], predicted_final_status
1 -->prob= [0.5208661622707759,0.47913383772922413],predicted_final_status: 0
1 -->prob= [0.2707403751493759,0.7292596248506242],predicted_final_status: 1
1 -->prob= [0.24573237208685889,0.7542676279131411],predicted_final_status: 1
1 -->prob= [0.4016747044218481,0.598325295578152],predicted_final_status: 1
0 -->prob= [0.861720398400684,0.13827960159931593],predicted_final_status: 0
1 -->prob= [0.48388018997943244,0.516119810

0 -->prob= [0.9364194126872407,0.06358058731275924],predicted_final_status: 0
1 -->prob= [0.15567622975583678,0.8443237702441632],predicted_final_status: 1
1 -->prob= [0.34260652974168226,0.6573934702583177],predicted_final_status: 1
0 -->prob= [0.5387568321791818,0.4612431678208182],predicted_final_status: 0
1 -->prob= [0.29297972424646257,0.7070202757535375],predicted_final_status: 1
1 -->prob= [0.09139746172600975,0.9086025382739903],predicted_final_status: 1
1 -->prob= [0.33080227303754905,0.669197726962451],predicted_final_status: 1
0 -->prob= [0.9713638881925395,0.028636111807460462],predicted_final_status: 0
1 -->prob= [0.33623740721728834,0.6637625927827117],predicted_final_status: 1
1 -->prob= [0.12123515711514699,0.878764842884853],predicted_final_status: 1
0 -->prob= [0.9857124205708114,0.014287579429188601],predicted_final_status: 0
0 -->prob= [0.9857124205708114,0.014287579429188601],predicted_final_status: 0
0 -->prob= [0.21500297956953035,0.7849970204304697],predicted_fi

0 -->prob= [0.527090098903583,0.47290990109641695],predicted_final_status: 0
0 -->prob= [0.9870191659773052,0.012980834022694685],predicted_final_status: 0
0 -->prob= [0.7669878467191217,0.2330121532808782],predicted_final_status: 0
0 -->prob= [0.9870191659773052,0.012980834022694685],predicted_final_status: 0
0 -->prob= [0.5181994228496968,0.48180057715030317],predicted_final_status: 0
0 -->prob= [0.5158047106853481,0.48419528931465194],predicted_final_status: 0
0 -->prob= [0.9870191659773052,0.012980834022694685],predicted_final_status: 0
0 -->prob= [0.9870191659773052,0.012980834022694685],predicted_final_status: 0
0 -->prob= [0.9870191659773052,0.012980834022694685],predicted_final_status: 0
0 -->prob= [0.9870191659773052,0.012980834022694685],predicted_final_status: 0
0 -->prob= [0.9870191659773052,0.012980834022694685],predicted_final_status: 0
0 -->prob= [0.9870191659773052,0.012980834022694685],predicted_final_status: 0
0 -->prob= [0.9870191659773052,0.012980834022694685],predi

0 -->prob= [0.9847136739503318,0.015286326049668234],predicted_final_status: 0
0 -->prob= [0.9847136739503318,0.015286326049668234],predicted_final_status: 0
0 -->prob= [0.9847136739503318,0.015286326049668234],predicted_final_status: 0
0 -->prob= [0.9739648246350218,0.026035175364978274],predicted_final_status: 0
0 -->prob= [0.9847136739503318,0.015286326049668234],predicted_final_status: 0
0 -->prob= [0.9328708445808905,0.06712915541910941],predicted_final_status: 0
1 -->prob= [0.29703094081189596,0.702969059188104],predicted_final_status: 1
1 -->prob= [0.2250560976369532,0.7749439023630467],predicted_final_status: 1
0 -->prob= [0.9613578049601591,0.03864219503984098],predicted_final_status: 0
0 -->prob= [0.9613578049601591,0.03864219503984098],predicted_final_status: 0
0 -->prob= [0.9835331465509103,0.01646685344908964],predicted_final_status: 0
1 -->prob= [0.29703094081189596,0.702969059188104],predicted_final_status: 1
0 -->prob= [0.6483914700325712,0.35160852996742875],predicted_

1 -->prob= [0.3439387919080661,0.6560612080919339],predicted_final_status: 1
1 -->prob= [0.3439387919080661,0.6560612080919339],predicted_final_status: 1
1 -->prob= [0.3007722333109306,0.6992277666890694],predicted_final_status: 1
1 -->prob= [0.2323780623731814,0.7676219376268185],predicted_final_status: 1
1 -->prob= [0.19042985057566747,0.8095701494243326],predicted_final_status: 1
0 -->prob= [0.9902599218183044,0.009740078181695653],predicted_final_status: 0
0 -->prob= [0.8610421221423804,0.13895787785761957],predicted_final_status: 0
1 -->prob= [0.3961932845935367,0.6038067154064632],predicted_final_status: 1
0 -->prob= [0.9902599218183044,0.009740078181695653],predicted_final_status: 0
1 -->prob= [0.3961932845935367,0.6038067154064632],predicted_final_status: 1
0 -->prob= [0.9669040528281316,0.03309594717186841],predicted_final_status: 0
0 -->prob= [0.3961932845935367,0.6038067154064632],predicted_final_status: 1
1 -->prob= [0.3961932845935367,0.6038067154064632],predicted_final_st

...

0 -->prob= [0.9613446372111495,0.038655362788850484],predicted_final_status: 0
0 -->prob= [0.6595374069804872,0.34046259301951276],predicted_final_status: 0
0 -->prob= [0.29478619129633915,0.7052138087036608],predicted_final_status: 1
1 -->prob= [0.3799204065827473,0.6200795934172527],predicted_final_status: 1
1 -->prob= [0.377685994115465,0.6223140058845349],predicted_final_status: 1
1 -->prob= [0.31822547919724026,0.6817745208027597],predicted_final_status: 1
1 -->prob= [0.292551778829057,0.707448221170943],predicted_final_status: 1
0 -->prob= [0.9613446372111495,0.038655362788850484],predicted_final_status: 0
0 -->prob= [0.9485648805290138,0.05143511947098624],predicted_final_status: 0
1 -->prob= [0.292551778829057,0.707448221170943],predicted_final_status: 1
0 -->prob= [0.9601641098117281,0.0398358901882719],predicted_final_status: 0
0 -->prob= [0.9708886961910659,0.029111303808934084],predicted_final_status: 0
0 -->prob= [0.9708886961910659,0.029111303808934084],predicted_final_st

0 -->prob= [0.9908999932617268,0.009100006738273103],predicted_final_status: 0
0 -->prob= [0.9908999932617268,0.009100006738273103],predicted_final_status: 0
0 -->prob= [0.9908999932617268,0.009100006738273103],predicted_final_status: 0
0 -->prob= [0.9908999932617268,0.009100006738273103],predicted_final_status: 0
0 -->prob= [0.4328335166120686,0.5671664833879314],predicted_final_status: 1
1 -->prob= [0.8527138067789377,0.14728619322106226],predicted_final_status: 0
0 -->prob= [0.9897194658623055,0.010280534137694516],predicted_final_status: 0
0 -->prob= [0.8743992774263409,0.12560072257365915],predicted_final_status: 0
1 -->prob= [0.14897940239088422,0.8510205976091157],predicted_final_status: 1
0 -->prob= [0.9908999932617268,0.009100006738273103],predicted_final_status: 0
0 -->prob= [0.9838049260025026,0.016195073997497405],predicted_final_status: 0
0 -->prob= [0.776327672660626,0.22367232733937398],predicted_final_status: 0
0 -->prob= [0.36831464123330326,0.6316853587666966],predict

0 -->prob= [0.9744256485223344,0.025574351477665618],predicted_final_status: 0
0 -->prob= [0.9744256485223344,0.025574351477665618],predicted_final_status: 0
0 -->prob= [0.9744256485223344,0.025574351477665618],predicted_final_status: 0
0 -->prob= [0.9744256485223344,0.025574351477665618],predicted_final_status: 0
0 -->prob= [0.9744256485223344,0.025574351477665618],predicted_final_status: 0
0 -->prob= [0.9744256485223344,0.025574351477665618],predicted_final_status: 0
0 -->prob= [0.9744256485223344,0.025574351477665618],predicted_final_status: 0
0 -->prob= [0.9744256485223344,0.025574351477665618],predicted_final_status: 0
0 -->prob= [0.9744256485223344,0.025574351477665618],predicted_final_status: 0
0 -->prob= [0.9744256485223344,0.025574351477665618],predicted_final_status: 0
0 -->prob= [0.9744256485223344,0.025574351477665618],predicted_final_status: 0
0 -->prob= [0.9744256485223344,0.025574351477665618],predicted_final_status: 0
0 -->prob= [0.9744256485223344,0.025574351477665618]

0 -->prob= [0.7144236835704094,0.2855763164295907],predicted_final_status: 0
0 -->prob= [0.9965912677257063,0.003408732274293802],predicted_final_status: 0
0 -->prob= [0.31518475421692616,0.6848152457830738],predicted_final_status: 1
0 -->prob= [0.9150675364918035,0.08493246350819643],predicted_final_status: 0
0 -->prob= [0.9965912677257063,0.003408732274293802],predicted_final_status: 0
0 -->prob= [0.6131626006839205,0.3868373993160795],predicted_final_status: 0
1 -->prob= [0.18503995792872796,0.8149600420712722],predicted_final_status: 1
0 -->prob= [0.9965912677257063,0.003408732274293802],predicted_final_status: 0
0 -->prob= [0.6944001903653798,0.3055998096346203],predicted_final_status: 0
1 -->prob= [0.26470445922147234,0.7352955407785278],predicted_final_status: 1
0 -->prob= [0.9954107403262847,0.004589259673715212],predicted_final_status: 0
1 -->prob= [0.16248554731923673,0.8375144526807633],predicted_final_status: 1
1 -->prob= [0.605803419234556,0.394196580765444],predicted_fina

0 -->prob= [0.9971366465275169,0.0028633534724829936],predicted_final_status: 0
0 -->prob= [0.9971366465275169,0.0028633534724829936],predicted_final_status: 0
0 -->prob= [0.9959561191280957,0.004043880871904406],predicted_final_status: 0
0 -->prob= [0.9156129152936143,0.08438708470638563],predicted_final_status: 0
1 -->prob= [0.16439541496875926,0.8356045850312407],predicted_final_status: 1
0 -->prob= [0.9156129152936143,0.08438708470638563],predicted_final_status: 0
0 -->prob= [0.8878168380680572,0.11218316193194278],predicted_final_status: 0
0 -->prob= [0.9971366465275169,0.0028633534724829936],predicted_final_status: 0
1 -->prob= [0.16439541496875926,0.8356045850312407],predicted_final_status: 1
0 -->prob= [0.9971366465275169,0.0028633534724829936],predicted_final_status: 0
0 -->prob= [0.9181675715080335,0.0818324284919665],predicted_final_status: 0
0 -->prob= [0.3133046341996001,0.6866953658003999],predicted_final_status: 1
0 -->prob= [0.16439541496875926,0.8356045850312407],predi

In [27]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(Predictions)
print("Test Error = %g" % (1.0 - accuracy))
print("Test Accuracy = %g" % accuracy)

rfModel = PipelineModel.stages[2]
print(rfModel)  # summary only

# # Transfer final_status(dataframe) into RDD
# ori_result = testData.rdd.map(lambda p: float(p.final_status))
# pre_result = Predictions.rdd.map(lambda p: float(p.final_status))
# # print(ori_result)  #RDD
# # print(pre_result)  #RDD
# labelsAndPredictions = ori_result.zip(pre_result)
# print(labelsAndPredictions.take(10))

# # first metrics
# from pyspark.mllib.evaluation import BinaryClassificationMetrics
# metrics = BinaryClassificationMetrics(labelsAndPredictions)
 
# # Area under precision-recall curve
# print("Area under PR = %s" % metrics.areaUnderPR)
 
# # Area under ROC curve
# print("Area under ROC = %s" % metrics.areaUnderROC)

# # second metrics
# accuracyTest = 1.0 * labelsAndPredictions.filter(lambda xv: xv[0] == xv[1]).count() / testData.count()
# print ("Test Accuracy For Random Forest = "  + str(accuracyTest) + "\n")


Test Error = 0.131646
Test Accuracy = 0.868354
RandomForestClassificationModel (uid=RandomForestClassifier_4d1a95c3d2d5ba64d674) with 10 trees
